<a href="https://colab.research.google.com/github/gretiere545/corpus/blob/main/CorpusPipeAirtableExtract_v1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Corpus Airtable API : Extract

In [1]:
#!/usr/bin/env python
# -*- coding: utf8 -*-
!pip install gspread-formatting
!pip install airtable-python-wrapper
import pandas as pd
import numpy as np
import requests
import json
import uuid
pd.set_option("display.width",1000)
# Ce bout de code pour pouvoir downloader des fichiers sur le drive
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Trad-Union/Corpus/ASAMLA
from google.colab import files

# Ce bout de code pour pouvoir downloader des google sheets dans des dataframes
from google.colab import auth
auth.authenticate_user()
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from gspread_formatting import *
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())
from airtable import Airtable
!export PYTHONIOENCODING=utf8


Mounted at /content/drive
/content/drive/MyDrive/Trad-Union/Corpus/ASAMLA


In [2]:
def get_cc_config(f):
  vk = []
  try:
    with open(f) as vk_f:
        vk = json.load(vk_f)
  except BaseException as e:
    print(e)
  return vk

def set_cc_config (vk, f):
  # sauvegarde du fichier de paramétrage
  with open(f, 'w') as fp:
      json.dump(vk, fp)  

In [3]:
#
# Ouverture de la Sheet Centrale (corpus_central_base)
#
def get_CCDB_wb(uri):
  sheet_central = uri
  wb_central = gc.open_by_url(sheet_central)
  return wb_central

def get_CCDB_data(wb, idx):
  t_corpus = wb.get_worksheet(idx)
  data_t_corpus = t_corpus.get_all_values()
  return data_t_corpus

#
# DataFrame CC global (toutes les langues)
#
def get_ccdf_global(CCDB):
  return pd.DataFrame(CCDB[1:], columns=CCDB[0])

#
# DataFrame CC Français (invariable)
#
def get_ccdf_fr(df_cc):
  # on supprime les doublons
  return df_cc[['uid','expression','theme','état','date','commentaires','index']]

In [4]:
# Config Langues
config_languages = 'med_vac_synthese.json'
vk_languages = get_cc_config(config_languages)

In [5]:
# Ouverture de la Sheet Centrale
sheet = 'https://docs.google.com/spreadsheets/d/1j9mtvKJn0Ad3FqxoJXMSKLJOfYTDOhtw3wxXZ4sQAIc'
wb  = gc.open_by_url(sheet)

In [6]:
# Config Corpus (de consolidation)
config_corpus_central = "corpus_central_config.json"
vk_corpus_central = get_cc_config(config_corpus_central)

In [7]:
# Base Centrale
# on charge l'onglet 1 (français)
db_cc_wb = get_CCDB_wb(vk_corpus_central[0]["uri"])
db_cc = get_CCDB_data (db_cc_wb, 1)
df_cc_global = get_ccdf_global(db_cc)
df_cc_fr = get_ccdf_fr(df_cc_global).drop_duplicates()

In [8]:
df_cc_global

,uid,expression,date,theme,état,commentaires,index,ams,eng,tur,rus,ukr,rou,hun,tig,alb,geo,arm,dar,pst,prs,aze,esp,amh,all,pol,som
0,f397cda1,A jeun,,Vaccination,,,A,صائم,With an empty stomach,Aç karnına,Натощак,Натщесерце,Pe stomacul gol,ÉHgyomorra,አብ ጥራሕ ከብድኻ(ኺ),Me stomak te zbrazet,უზმოზე,Սոված փորին,گرسنه,په خالي معدې سره,گرسنه,,En ayunas,,,Na czczo,
1,f9bc13ef,À Quelle moment de la journée prenez-vous le m...,,Médecine générale,,,À,في أي وقت من اليوم تتناول الدواء؟,,,,,,,,,,,,,,,,,,O jakiej porze dnia przyjmujesz lek ?,
2,6e65dde7,Abcès,,Médecine générale,,,A,خُرّاج,,Apse,,,Abces,,,,,,,,,,,,,Ropień,
3,95c4a863,Accès fébrile concomitant chez un autre membre...,,Vaccination,,,A,نوبة حمى مزامنة عند فرد اخر من الأسرة,Concomitant febrile attack in an other family ...,,Сопровождающий жар у другого члена семьи ?,Супроводжуючий жар у іншого члена сім'ї ?,Mai are cineva temperatură în familie ?,Lázas valaki a családban ?,ኣብ ፋሚሊ ሓደ ዓይነት ጸገም ዶ ኣሎኩም,Sulm shoqërues i etheve në një pjesëtar tjetër...,ოჯახის წევრები თუ არიან ამით დაავადებული,Ուղեկցող ջերմություն տան անդամներից մեկի մոտ,حمله تب دار در یکی دیگر از اعضای خانواده؟,د کورنۍ په بل غړي باندې د بخار برید؟,حمله تب دار در یکی دیگر از اعضای خانواده؟,,Acceso febril concomitante en otro miembro de ...,,,,
4,30579682,Accident,,Vaccination,,,A,حادث,Accident,Kaza,Несчастный случай,Нещасний випадок,Accident,Baleset,ሐደጋ,Aksident,უბედური შემთხვევა,Վթար,تصادف,حادثه,تصادف,,Accidente,,,Wypadek,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,52418,Vous avez des allergies aux médicaments ?,,Médecine générale,,,V,,,,,,,,,,,,,,,,,,,,
251,5baecf7f,Vous avez mal autre part ?,,Médecine générale,,,V,هل لديك ألم في مكان آخر؟,,Başka bir yeriniz ağrıyormu ?,,,Aveti dureri in alta parte ?,,,,,,,,,,,,,Czy boli Cię coś gdzieś indziej ?,
252,36e66161,Y a-t-il des cas contagieux à l'école ?,,Vaccination,,,Y,هل توجد حالات معدية في المدرسة؟,Are there contagious cases at school ?,Okulda blaşıcı durum varmı ?,Бывают ли в школе заразные случаи ?,Чи бувають у школі заразні випадки ?,La sșcoală sunt cazuri de infecție contagioasă ?,Aktualizálták / Frissitették a védőoltásokat ?,አብ ትምህርቲ ዲዩ ለጊብካ ?,A ka raste ngjitëse në shkollë ?,თუ არის ავადმყოფობის შემთხვევა სკოლაში ?,Վարակիչ դեպքեր կա՞ն դպրոցում,آیا موارد مسری در مدرسه وجود دارد؟,ایا په ښوونځي کې کومې ساري پیښې شتون لري؟,آیا موارد مسری در مدرسه وجود دارد؟,,¿Tienen casos contagiosos en la escuela ?,,,Czy w szkole są przypadki zakaźne ?,
253,c32f5fd9,Y a-t-il des maladies chroniques dans votre fa...,,Médecine générale,,,Y,هل يوجد أمراض مزمنة في عائلتك؟,,Ailenizde kronik hastalık varmı ?,,,Sunt persoane in familie cu boli cronice ?,,,,,,,,,,,,,Czy w Twojej rodzinie są jakieś choroby przewl...,


### Appel de l'API
https://towardsdatascience.com/downloading-airtable-data-into-python-89e5c7107a24

In [9]:
params = ()
api_key = "keyrI98TIqu6mbFcf" 
api_key_gmail = "keysgpJWq3aWwqSZe"
headers = {"Authorization": "Bearer " + api_key,"Content-Type" : "application/json"}
base_id = "appawT3gg7cJhPYv0" # Base Nouvelle HV
table_name = 'Theme'
airtable_theme = Airtable(base_id, table_name, api_key)

In [10]:
table_name = 'Corpus'
airtable_corpus = Airtable(base_id, table_name, api_key)

```
for page in airtable.get_iter(view='Grid view',sort='Proposition'):
  for record in page:
    value = record['fields']['Proposition']
```



In [34]:
vk_theme = airtable_theme.get_all(view='Grid view',sort='Terme')
df_vk_theme = pd.DataFrame.from_records((r['fields'] for r in vk_theme))

In [35]:
vk_corpus = airtable_corpus.get_all(view='Admin',sort='Proposition')
df_vk_corpus = pd.DataFrame.from_records((r['fields'] for r in vk_corpus))

In [36]:
# on caste la série en string
df_vk_theme["Corpus"] = df_vk_theme["Corpus"].apply(lambda x: x[0])
# on caste en string tout le reste aussi
df_vk_theme["Définition"] = df_vk_theme["Définition"].apply(lambda x: x[0])


In [37]:
# recherche de REC ID pour obtenir la traduction (ici on prend le russe)
def get_corpus_rec (key):
  return df_vk_corpus.loc[df_vk_corpus["Name ID"]==key]["rus"].values.item()

In [56]:
# recherche de REC ID pour obtenir la traduction (ici on prend le russe)
def get_phrase (key):
  r = ""
  if pd.notna(key):
    print(key)
    r = df_vk_corpus.loc[df_vk_corpus["Name ID"]==key]["Proposition"].values.item()
  return r 


In [39]:
# On ajoute la traduction
df_vk_theme["rus"] = df_vk_theme["Corpus"].apply(lambda x: get_corpus_rec(x))

In [45]:
df_vk_theme = df_vk_theme.explode("Phrase")

In [61]:
df_vk_theme = df_vk_theme.explode("Thème")

In [65]:
df_vk_theme = df_vk_theme.explode("Expression")

In [67]:
df_vk_theme["Expression"] = df_vk_theme["Expression"].apply(lambda x: get_phrase(x))

recDNzt9YOceP1MSY
recMyIOGoTKuZ55Si


In [ ]:
df_vk_theme["Phrase"] = df_vk_theme["Phrase"].apply(lambda x: get_phrase(x))

In [68]:
df_vk_theme

,Corpus,Rubrique,Terme,Définition,Thème,Phrase,Expression,rus
0,recVvIHdNWf80zvsG,Traitements,À quel moment de la journée vous prenez le méd...,phrase,Médecine générale,,,NaN
1,recFclp4FIK5u5sa3,Généralités - état civil,Age,nm,Médecine générale,Quel âge avez-vous ?,,NaN
2,reccvrUrXdeQgVAnC,Antécédents,Antécédents,nm,Vaccination,,,ПредшествующиE заболевания
3,reczaubMtqEVNLdn3,Antécédents,Avez-vous déjà été opéré(e) ?,phrase,Vaccination,,,Вам когда-нибудь делали операцию ?
4,reckZlBAzDSte2Aql,Antécédents,Avez-vous déjà été transfusé(e) ?,phrase,Vaccination,,,Вам когда-нибудь делали переливание крови ?
5,recMenxhf0pi6hyRb,Antécédents,Avez-vous des antécédents dans la famille ?,phrase,Vaccination,D'autres membres de la famille sont-ils actuel...,,У вас есть предшествующиE заболевания семье (а...
6,rec33YHbQfJVyNqZB,Le corps humain,Bras,nm,Vaccination,Montrez la trace au niveau du bras,,РукA
7,rec8pS46MYGy4eNMk,Examen,Fibroscopie,nf,Médecine générale,,Fibroscopie digestive,NaN
8,recN8cQKAYrainO3w,Symptômes,Fièvre,nf,Vaccination,Avez-vous de la fièvre ?,,"Жар, горячка"
8,recN8cQKAYrainO3w,Symptômes,Fièvre,nf,Vaccination,Date du début de la fièvre,,"Жар, горячка"


In [62]:
gk = df_vk_theme.groupby("Rubrique")

In [63]:
gk.get_group('Etat général')

,Corpus,Rubrique,Terme,Définition,Thème,Phrase,Expression,rus
13,recnFsFSoE1DCjYqt,Etat général,Poids,nm,Vaccination,Votre poids a-t-il augmenté ?,NaN,Вес
13,recnFsFSoE1DCjYqt,Etat général,Poids,nm,Vaccination,Votre poids a-t-il baissé ?,NaN,Вес
13,recnFsFSoE1DCjYqt,Etat général,Poids,nm,Vaccination,Votre poids est-il stable ?,NaN,Вес
13,recnFsFSoE1DCjYqt,Etat général,Poids,nm,Vaccination,Quel est votre poids ?,NaN,Вес
16,recPd4zs5SiIqkQXA,Etat général,Taille,nf,Vaccination,Quelle est votre taille ?,NaN,Рост


In [53]:
df_vk_corpus.loc[df_vk_corpus["Name ID"]=="recNQqTXnpEEJAQm6"]["Proposition"].values.item()

'Quelle est votre taille\u202f?'

recXflm2lWveR6neX
recJ31CEHNRSc4H6m
rec3KIsjsQ4hN4yj3
recsRiPxYQHWzQ55y
recG6Td4yr6cOrYXL
recDIZedc9PyD4aY4
recY3xPGlPI4PuVjE
receaQEuvP4z54rkk
recpJHzSrc1ertCTr
recuyWKo1a0ujlY7U
recnEzvTeb7iJiZYH
recNQqTXnpEEJAQm6
recyhlDdiBICVS3zs


END

Recherche des UID manquants dans Airtable

In [ ]:
def airtable_get_missing_uids(df):
  return df.loc[pd.isna(df["UID"])]


Recherche dans Airtable par Séquence

In [ ]:
def airtable_get_records_by_sequence(df, sequence):
  return df.loc[df["Séquence"]==sequence]


In [ ]:
df_vkat.loc[df_vkat["Séquence"]=="1. Présentation + antécédents"]

In [ ]:
airtable_get_missing_uids(df_vkat)

Rceherche des termes présents dans Central, présents dans AIRTBL mais sans UID

In [ ]:
df_cc_fr

In [ ]:
vk_at_trad = airtable.get_all(view='Traductions',sort='Proposition')
df_vk_at_trad = pd.DataFrame.from_records((r['fields'] for r in vk_at_trad))
df_vk_at_trad = df_vk_at_trad[["Proposition","idx_cor","UID"]]
df_vk_at_trad_notna = df_vk_at_trad.loc[pd.notna(df_vk_at_trad["UID"])]
df_vk_at_trad_na = df_vk_at_trad.loc[pd.isna(df_vk_at_trad["UID"])]

### merge côté AT : avec les UID présents 

In [ ]:
df_merged = pd.merge(df_cc_global, df_vk_at_trad_notna, how="right", left_on='uid', right_on='UID')

## On complète avec les traductions

In [ ]:
for index, row in df_merged.iterrows():
  if row["uid"] is not None:
    record = airtable.match('UID', row["uid"])
    aDict = {}
    for i in vk_languages:
      key = i["trigramme"]
      aDict[key] = row[key]
      #print (record['id'] +" " + row["eng"])
    airtable.update(record['id'], aDict)


## On complète avec les UID manquants

In [ ]:
df_vk_at_trad_na

In [ ]:
df_merged = pd.merge(df_cc_global, df_vk_at_trad_na, how="right", left_on='expression', right_on='Proposition')

In [ ]:
df_merged = df_merged.loc[pd.notna(df_merged["uid"])]

In [ ]:
for index, row in df_merged.iterrows():
  if row["uid"] is not None:
    record = airtable.match('Proposition', row["expression"])
    aDict = {}
    key = "UID"
    aDict[key] = row["uid"]
    for i in vk_languages:
      key = i["trigramme"]
      aDict[key] = row[key]
      #print (record['id'] +" " + row["eng"])
    airtable.update(record['id'], aDict)

## On vide toute la base AT dans Central Consolidation

In [ ]:
vk_at = airtable.get_all(view='Admin',sort='Proposition')
df_vkat = pd.DataFrame.from_records((r['fields'] for r in vk_at))

In [ ]:
# Corpus_consolidation
uri = "https://docs.google.com/spreadsheets/d/1cSnSnnQunL-I7hpyYkIqbtuGr55unVJ3Qe8URSrjTr8/edit?usp=sharing"

wb = get_CCDB_wb(uri)
db_cc = get_CCDB_data (wb, 0)

nom_onglet = 'Termes'
# si l'onglet existe déjà
try:
  ws = wb.worksheet(nom_onglet)
  wb.del_worksheet(ws)
except:
  print ("Onglet inexistant !")

wb.add_worksheet(nom_onglet, 1, 1)
export_sheet = wb.worksheet(nom_onglet)
set_with_dataframe(export_sheet, df_vkat)



END

In [ ]:
record = airtable.match('UID', '30579682')
fields = {'eng': 'Accident'}
airtable.update(record['id'], fields)

{'createdTime': '2021-12-10T08:47:48.000Z',
 'fields': {'CATEGRAM': ['nm'],
  'Name ID': 'recGaRQGjV5qN3QBN',
  'Proposition': 'Accident',
  'TYPE': 'vocabulaire',
  'Thématique': 'Vaccination',
  'UID': '30579682',
  'VOCABULAIRE': 'Général',
  'eng': 'Accident',
  'idx': 'A',
  'idx_cor': 'A'},
 'id': 'recGaRQGjV5qN3QBN'}

In [ ]:
record

{'createdTime': '2021-12-10T08:47:48.000Z',
 'fields': {'CATEGRAM': ['nm'],
  'Name ID': 'recGaRQGjV5qN3QBN',
  'Proposition': 'Accident',
  'TYPE': 'vocabulaire',
  'Thématique': 'Vaccination',
  'UID': '30579682',
  'VOCABULAIRE': 'Général',
  'idx': 'A',
  'idx_cor': 'A'},
 'id': 'recGaRQGjV5qN3QBN'}

In [ ]:
def airtable_table_get(uri, params, headers):
  airtable_records = []
  run = True
  while run is True:
    response = requests.get(uri, params=params, headers=headers)
    airtable_response = response.json()
    airtable_records += (airtable_response['records'])
    if 'offset' in airtable_response:
      run = True
      params = (('offset', airtable_response['offset']),)
    else:
      run = False
  return airtable_records

# Table Corpus

In [ ]:

url = "https://api.airtable.com/v0/" + base_id + "/" + table_name


### On lit tous les enregistrements, puis on met dans un dataframe

In [ ]:
airtable_records = airtable_table_get(url, params, headers)

airtable_rows = [] 
airtable_index = []
for record in airtable_records:
  airtable_rows.append(record['fields'])
  airtable_index.append(record['id'])
df = pd.DataFrame(airtable_rows, index=airtable_index)
df = df [['Proposition','UID', 'Name ID', 'DEPENDANCE', 'idx', 'idx_cor','Statut','Thématique']]
df.sort_values(by=['Proposition'], ascending=True, inplace=True)

In [ ]:
df.head(10)

,Proposition,UID,Name ID,DEPENDANCE,idx,idx_cor,Statut,Thématique
recKbPHqDTXNPwYK0,A jeun,f397cda1,recKbPHqDTXNPwYK0,NaN,J,J,NaN,Vaccination
rec1qTnfWxjhx6SBl,AME - Aide médicale d'État,f49563a9,rec1qTnfWxjhx6SBl,NaN,A,A,NaN,Vaccination
rec0HgtD22F7TWM3W,Abcès,6e65dde7,rec0HgtD22F7TWM3W,NaN,A,A,NaN,Médecine générale
recGaRQGjV5qN3QBN,Accident,30579682,recGaRQGjV5qN3QBN,NaN,A,A,NaN,Vaccination
rec3MxDq67MC982ng,Accident cérébral,6f214e4c,rec3MxDq67MC982ng,[recGaRQGjV5qN3QBN],A,A,NaN,Vaccination
reciexfGbNKvXzQg8,Alcool,NaN,reciexfGbNKvXzQg8,NaN,A,A,NaN,Médecine générale
recfY4uKkgK8Jp8GO,Allergie,8b195cf4,recfY4uKkgK8Jp8GO,NaN,A,A,NaN,Vaccination
recSKSrXVLhDfMq6w,Allergie alimentaire,f754eebf,recSKSrXVLhDfMq6w,[recfY4uKkgK8Jp8GO],A,A,NaN,Vaccination
rectthHdPchXi1zr9,Allergie médicamenteuse,7256c169,rectthHdPchXi1zr9,"[recfY4uKkgK8Jp8GO, recOivQOmeragygVr]",A,A,NaN,Vaccination
recHNzXfvWWm0Lwp8,Allergie à un médicament,316fa7ab,recHNzXfvWWm0Lwp8,"[recfY4uKkgK8Jp8GO, recOivQOmeragygVr]",A,A,NaN,Vaccination


In [ ]:
def make_unique_id():
  return str(uuid.uuid4())[:8]

In [ ]:
def lookup_uid (uid):
  try:
    record = df_cc_fr.loc[df_cc_fr["uid"]==uid]["index"].item()
  except Exception as e:
    list_no_uid.append(uid)
    return uid
  if record is not None:
    return record

In [ ]:
x = df_cc_fr.loc[df_cc_fr["uid"]=="72f0e8db"]["index"].item()

In [ ]:
list_no_uid = [] # uid non présents côté central = mots à créer
df['UID'] = df['UID'].apply(lambda x: make_unique_id() if pd.isna(x) else x)
df.idx_cor = df['UID'].apply(lambda x: lookup_uid(x))

convert the dataframe to a dictionary of dictionaries.

In [ ]:
df.tail()

,Proposition,UID,Name ID,DEPENDANCE,idx,idx_cor,Statut,Thématique
recia1L6nICYg6LJ8,Zona,d3dba8e4,recia1L6nICYg6LJ8,NaN,NaN,Z,NaN,Vaccination
recVvIHdNWf80zvsG,À quelle moment de la journée vous prenez le m...,0f86d1ef,recVvIHdNWf80zvsG,[recOivQOmeragygVr],M,0f86d1ef,Ramona,Médecine générale
recWp0rajm2qKcJuF,Êtes vous vacciné (e) ?,c5df267a,recWp0rajm2qKcJuF,NaN,NaN,Ê,NaN,Médecine générale
recBY5NXGM8UtBcaX,Êtes-vous enceinte ?,7768882c,recBY5NXGM8UtBcaX,NaN,NaN,Ê,NaN,Médecine générale
reczy8Os5Hc3qZ1yA,Œdème,72f0e8db,reczy8Os5Hc3qZ1yA,NaN,NaN,Œ,NaN,Médecine générale


In [ ]:
dict_records = df.tail().to_dict(orient="index")

In [ ]:
for record_id, upload_data in dict_records.items():
  record_url = url + "/" + record_id
  upload_dict = {"records" : [{"fields" : upload_data}]}
  upload_json = json.dumps(upload_dict)
  requests.post(url, data=upload_json, headers=headers)

In [ ]:
headers

{'Authorization': 'Bearer keyrI98TIqu6mbFcf',
 'Content-Type': 'application/json'}

In [ ]:
requests.patch(url, data=upload_json, headers=headers)

<Response [422]>

In [ ]:
upload_json='{"records": [{"fields": {"Proposition": "OEDEME", "id": "reczy8Os5Hc3qZ1yA"}}]}'

In [ ]:
requests.post(record_url, data=upload_json, headers=headers)

<Response [404]>

In [ ]:
def fmt_table(wb, nom_onglet):
  # onglet
  ws = wb.worksheet(nom_onglet)

  # Format de la partie gauche
  fmt = cellFormat(
      backgroundColor=color(0.91, 0.96, 0.93),
      textFormat=textFormat(bold=False, foregroundColor=color(0,0,0), fontSize='10'),
      horizontalAlignment='LEFT'
      )
  format_cell_range(ws, 'A:', fmt)

  # format de l'entete
  fmt = cellFormat(
      backgroundColor=color(0.7725,0.8431,0.7922),
      textFormat=textFormat(bold=True, foregroundColor=color(0,0,0), fontSize='10'),
      horizontalAlignment='LEFT'
      )
  format_cell_range(ws, '1', fmt)
  
  # format de l'auto wrap
  fmt = cellFormat(
      wrapStrategy='WRAP'
      )
  format_cell_range(ws, 'B:C', fmt)

  # on gèle l'entete et les colonnes de gauche
  set_frozen(ws, rows=1, cols=1)
  set_column_width(ws, 'A:C', 500)


In [ ]:
nom_onglet = 'Liste'
# si l'onglet existe déjà
try:
  ws = wb.worksheet(nom_onglet)
  wb.del_worksheet(ws)
except:
  print ("Onglet inexistant !")

wb.add_worksheet(nom_onglet, 1, 1)
export_sheet = wb.worksheet(nom_onglet)
set_with_dataframe(export_sheet, df)
fmt_table(wb,nom_onglet)


In [ ]:
table_name = 'Structure'
url = "https://api.airtable.com/v0/" + base_id + "/" + table_name

In [ ]:
df

,Name,Situation de départ,Notes
recnI80hvydv71fs4,0.Conception,"Plus on possède, plus on est soi-même possédé....","Trouver le plaisir d’écrire. Partir d’idées, d..."
recI2pgkNq2iP52rm,1.Situation initiale,Un jour d'été où il pleut et fait un temps abo...,Une vengeance d'une femme éconduite lors d'un ...
recb2DqhB9GvfI12n,2.Personnage,Un homme (marié ?) qui a donné rendez-vous à u...,Un Dom Juan sûr de son charme et imbu de sa pe...
recNLdU01JpCnvknJ,3.Acte déclencheur,La femme lui propose un jeu. Elle est dans le ...,NaN
recRxZVhpSPD6oQXx,4.Réaction,L'homme est d'abord tenté de partir car il est...,NaN
recRr06fOM4hGGaLY,5.Péripéties,"Pendant la visite (ennuyeuse), l'homme tente d...",NaN
recvWnAYn9VNySVlB,6.Chute,Un spectacle où une partie du public est appel...,NaN
recDmZtMMNmiy7R0M,Récit,NaN,NaN
